In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient

from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacUser"
password = "abc123"
shelter = AnimalShelter(password, username)

#create the dataframe
df = pd.DataFrame.from_records(shelter.getRecordCriteria({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('BHobbs\' SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #allow a row to be selected
        selected_rows=[],
        filter_action="native", #allow a filter
        sort_action="native", #allow sorting
        page_action="native", #enable pagination
        page_current=0, #set start page
        page_size=10, #set rows per page

    ),
    html.Br(),
     html.Hr(),
     html.Div( #div for the map
            id='map-id',
            className='col s12 m6',
            )
    ])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(virtualRows):
    
    print(virtualRows)

    #austin Texas is [30.75, -97.48]
    
    #create the views
    if not virtualRows: #build a default view if there are no selected lines
        markerArray = (30.75,-97.48) #default marker at Austin Animal Shelter
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
        
    else: #build the contextual views based on the selection
        dff = pd.DataFrame(df.iloc[virtualRows]) #convert the datatable to a dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) #strip out the lat
        coordLong = float(dff['location_long'].to_string().split()[1]) #strip out the long
        markerArray = (coordLat, coordLong) #build the array based on selection
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    return [dl.Map(style={'width': '600px', 'height': '300px'}, center=[30.75,-97.48],
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]
    
app